## Representation learning for NLP

### Préparation des données
Chargement du dataset et mappage des labels en entiers (0: Contradiction, 1: Neutre, 2: Conséquence).

### Ajustement des encodeurs (classification multi-classes)
- **Objectif :** Résoudre la tâche en utilisant LoRA
- **Modèle :** CamemBERT 2.0 et CamemBERTa 2.0
- **Étapes :** (pour chaque modèle)
    * Chargement du modèle
    * Configurer LoRA aux modules query/value
    * Entraînement : Ajuster par apprentissage supervisé sur la tâche de classification.

### Ajustement des décodeurs (génératif)
- **Objectif :** Résoudre la tâche en utilisant un modèle génératif (LLM) avec LoRA.
- **Modèles :** Llama 3 (modèle 1B ou 3B).
- **Étapes :**
    * Formatage : Convertir le jeu de données en format de prompt (ex: "Prémisse : X. Hypothèse : Y. Relation : [MASK]").
    * Config LoRA : Appliquer LoRA au modèle Llama 3.
    * Entraînement : Ajuster par apprentissage supervisé (Causal Language Modeling).
    * Évaluation : Générer des réponses sur le jeu de test et mapper le texte de sortie aux étiquettes.

### Apprentissage en contexte (prompt engineering)
- **Objectif :** Évaluer Llama 3 (3B ou 8B) sans mettre à jour les poids (Inférence seule).
- **Expériences :**
    - **0-shot :** Demander au modèle de classer sans exemples.
    - **Few-shot :** Fournir quelques exemples de prémisse/hypothèse/étiquette avant de demander la prédiction.
    - **Chain-of-Thought (CoT) :** Prompter le modèle pour qu'il explique pourquoi il a choisi une étiquette. Cela répond à l'objectif pédagogique de compréhension du CoT.

### Évaluation des modèles
- matrice de confusion et métriques standard pour la classification multi-classes: accuracy, F1-score, weighted F1-score, precision, recall

# Introduction

Depuis la publication de l'article *"Attention Is All You Need"* (Vaswani et al., 2017), le traitement du langage naturel (NLP) a connu une transformation majeure. Les approches précédentes, comme **Bag-of-Words** ou **TF-IDF**, représentaient les mots par comptage de fréquence, tandis que **Word2Vec** avait permis une avancée significative en créant des embeddings capturant certaines relations sémantiques. Cependant, ces modèles présentaient une limitation fondamentale : l'absence de prise en compte du **contexte**.

En effet, les embeddings statiques ne capturaient ni la **position des mots** ("Pierre mange une pizza" diffère de "Une pizza mange Pierre"), ni la **polysémie** (le mot "modèle" désigne un concept mathématique ou une profession selon le contexte). Pour résoudre ces limitations, Vaswani et al. ont introduit l'architecture **Transformer**, reposant sur un mécanisme de **self-attention** qui permet de capturer les dépendances entre tous les mots d'une séquence. Contrairement aux approches séquentielles (RNN, LSTM), cette architecture permet une parallélisation efficace des calculs sur GPU.

Le Transformer original comprend deux composants : un **encodeur** qui transforme le texte en représentations vectorielles contextualisées, et un **décodeur** qui génère du texte à partir de ces représentations via un mécanisme de cross-attention. Des architectures dérivées se sont ensuite spécialisées : **BERT** (Devlin et al., 2019) utilise une architecture *encoder-only* avec une attention bidirectionnelle pour des tâches de compréhension (classification, NER), tandis que **GPT** (Radford et al., 2018) utilise une architecture *decoder-only* avec du masked self-attention pour des tâches de génération (agents conversationnels comme ChatGPT ou Claude).

Ces modèles, souvent appelés **LLM** (Large Language Models), suivent généralement deux phases d'entraînement. Le **pré-entraînement** permet au modèle d'apprendre le langage de manière auto-supervisée sur de larges corpus, produisant un *foundation model*. Le **fine-tuning** adapte ensuite ce modèle à une tâche spécifique. Parmi les méthodes récentes, **LoRA** (Hu et al., 2021) permet un ajustement efficace en ne modifiant qu'une fraction des paramètres (~0.3%), rendant le fine-tuning accessible avec des ressources limitées.

Dans ce projet, nous abordons la tâche d'**inférence en langage naturel (NLI)** en français, qui consiste à déterminer la relation sémantique entre deux phrases : conséquence (*entailment*), contradiction ou neutralité. Cette tâche est fondamentale en NLP car elle évalue la capacité d'un modèle à raisonner sur le sens des phrases. Nous comparons plusieurs approches :

- **Encodeurs avec LoRA** : CamemBERT (architecture RoBERTa) et CamemBERTa (architecture DeBERTa)
- **Décodeurs avec LoRA** : Llama 3 (1B et 3B paramètres)
- **Apprentissage en contexte** : Llama 3 en modes zero-shot, few-shot et Chain-of-Thought